# Práctica 5: Implementación de Simplex

## Librerias

In [2]:
import numpy as np

## Simplex

Función donde se implementa el algoritmo Simplex para optimización de problemas de programación lineal en la forma estandar.

* El código fue adapatado de: https://github.com/mmolteratx/Simplex



In [3]:
# ------------------------------------------------------------------
#                      Funciones Auxiliares
# ------------------------------------------------------------------

def printTableau(tableau, f):
  
    f = f[np.nonzero(f)]
    print("ind \t", end = "")
    for j in range(0, len(f)):
        print("x_" + str(j), end = "\t")
    for j in range(0, (len(tableau[0]) - len(f) - 2)):
        print("s_" + str(j), end = "\t")
    
    print()
    for j in range(0, len(tableau)):
        for i in range(0, len(tableau[0])):
            if(not np.isnan(tableau[j, i])):
                if(i == 0):
                    print(int(tableau[j, i]), end = "\t")
                else:
                    print(round(tableau[j, i], 2), end = "\t")
            else:
                print(end = "\t")
        print()

def getTableau(c, A, b):
  # Construct starting tableau
  c[0:len(c)] = -1 * c[0:len(c)]

  t2 = np.array([None, 0])
  numVar = len(c)
  numSlack = len(A)
    
  t2 = np.hstack(([None], c, [0]))
  
  basis = np.array([0] * numSlack)
  
  for i in range(0, len(basis)):
      basis[i] = numVar + i
      
  t1 = np.hstack((np.transpose([basis]), A, np.transpose([b])))
  
  tableau = np.vstack((t1, t2))
  
  tableau = np.array(tableau, dtype ='float')
  
  return tableau

# ------------------------------------------------------------------
#                      Funcion Simplex
# ------------------------------------------------------------------

def simplex(f, A, b, print_iter=True):
  for i in range(len(f)):
    f[i] = -1 * f[i]
    
  # Build Tableu
  tableau = getTableau(f, A, b)
      
  if(print_iter):
      print("Starting Tableau:")
      printTableau(tableau, f)
  
  # Assume initial basis is not optimal
  optimal = False

  # Keep track of iterations for display
  iter = 1

  while not optimal:
      
    if print_iter:
      print("--------------------------------------------------------------")
      print("ITERATION ", iter)
      printTableau(tableau, f)
        
    # Look for direction of decreased cost
    for cost in tableau[-1, 1:-1]:
      if cost < 0:
        optimal = False
        break
      optimal = True

    # If all directions result in decreased cost SBF is optimal
    if optimal: 
      break
      
    # nth variable enters basis, account for tableau indexing
    n = tableau[-1, 1:-1].tolist().index(np.amin(tableau[-1, 1:-1])) + 1 # PUEDE QUE NO SEA 2

    # Minimum ratio test, rth variable leaves basis 
    minimum = 99999
    r = -1
    
    for i in range(0, len(tableau)-1): 
      if (tableau[i, n] > 0):
        val = tableau[i, -1]/tableau[i, n]
        if val<minimum: 
          minimum = val 
          r = i
                    
    pivot = tableau[r, n] 
    
    print("Pivot Column:", n)
    print("Pivot Row:", r)
    print("Pivot Element: ", pivot)

    # Perform row operations 
    # Divide the pivot row with the pivot element 
    tableau[r, 1:] = tableau[r, 1:] / pivot 
    
    # Pivot other rows
    for i in range(0, len(tableau)): 
      if i != r:
        mult = tableau[i, n] / tableau[r, n]
        tableau[i, 1:] = tableau[i, 1:] - mult * tableau[r, 1:] 

    # New basic variable 
    tableau[r, 0] = n - 1
    iter += 1
      
  if print_iter:
    print("--------------------------------------------------------------")
    print("Final Tableau reached in", iter, "iterations")
    printTableau(tableau, f)
  else:
      print("Solved")
      
  x = np.array([0] * len(f), dtype = float)
  # Save coefficients
  for key in range(0, (len(tableau))):
      if (tableau[key, 0] < len(f)):
          x[int(tableau[key, 0])] = tableau[key, -1]
  
  optimalValue = -1 * tableau[-1,-1]

  print("--------------------------------------------------------------")
  print("SOLUTION: ")
  print("Fobj Optimal Value: " + str(optimalValue))
  print("Solution: " + str(x))


##  Ejemplo

Ejemplo de como utilizar la función implementada en python.

Problema:
$$ \min f(x) = -3 x_1 -  x_2 - 3 x_3 $$

$$ \begin{array}{rl}
  2x_1 + x_2 + x_3 & \leq 2 \\
  x_1 + 2x_2 + 3x_3 & \leq 5 \\
  2x_1 + 2x_2 + x_3 & \leq 6 \\
  x_1,x_2, x_3 & \geq 0 \\
 \end{array}$$

 Forma Estándar:
 $$ \min f(x) = -3 x_1 -  x_2 - 3 x_3  $$

$$ \begin{array}{rl}
  2x_1 + x_2 + x_3 + y_1 & = 2 \\
  x_1 + 2x_2 + 3x_3 + y_2 & = 5 \\
  2x_1 + 2x_2 + x_3 + y_3 & = 6 \\
  x_1,x_2, x_3, y_1, y_2, y_3 & \geq 0 \\
 \end{array}$$

In [4]:
# Probar función
f = np.array([-3, -1, -3, 0, 0, 0])
A = np.array([[2, 1, 1, 1, 0, 0], [1, 2, 3, 0, 1, 0], [2, 2, 1, 0, 0, 1]])
b = np.array([2, 5, 6])

simplex(f,A,b)

Starting Tableau:
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
6	2.0	1.0	1.0	1.0	0.0	0.0	2.0	
7	1.0	2.0	3.0	0.0	1.0	0.0	5.0	
8	2.0	2.0	1.0	0.0	0.0	1.0	6.0	
	-3.0	-1.0	-3.0	0.0	0.0	0.0	0.0	
--------------------------------------------------------------
ITERATION  1
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
6	2.0	1.0	1.0	1.0	0.0	0.0	2.0	
7	1.0	2.0	3.0	0.0	1.0	0.0	5.0	
8	2.0	2.0	1.0	0.0	0.0	1.0	6.0	
	-3.0	-1.0	-3.0	0.0	0.0	0.0	0.0	
Pivot Column: 1
Pivot Row: 0
Pivot Element:  2.0
--------------------------------------------------------------
ITERATION  2
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
0	1.0	0.5	0.5	0.5	0.0	0.0	1.0	
7	0.0	1.5	2.5	-0.5	1.0	0.0	4.0	
8	0.0	1.0	0.0	-1.0	0.0	1.0	4.0	
	0.0	0.5	-1.5	1.5	0.0	0.0	3.0	
Pivot Column: 3
Pivot Row: 1
Pivot Element:  2.5
--------------------------------------------------------------
ITERATION  3
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
0	1.0	0.2	0.0	0.6	-0.2	0.0	0.2	
2	0.0	0.6	1.0	-0.2	0.4	0.0	1.6	
8	0.0	1.0	0.0	-1.0	0.0	1.0	4.0	
	0.0	1.4	0.0	1.2	0.6	0.0	5.4	
--------------------------------